In [63]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from urllib.error import URLError
from ssl import SSLError
import socket
import time
import random

In [149]:
# investing.com에서 얻은 차트 상위 200 개의 종목 정보
df_e = pd.DataFrame(pd.read_csv(f'./data/search_1.csv'))
for i in range(2, 5):
    df_s = pd.read_csv(f'./data/search_{i}.csv')
    df_e = pd.concat([df_e, df_s], ignore_index=True)

In [98]:
df = pd.DataFrame()
df['stock'] = df_e['종목']
df['trick'] = df_e['기호']

In [102]:
df

,stock,trick
0,삼성전자우,005935
1,삼성전자,005930
2,LG엔솔,373220
3,LG 화학,051915
4,SK하이닉스,000660
...,...,...
195,DL Holdings Prf,000215
196,현대위아,011210
197,SK케미칼,285130
198,에스엠,041510


In [104]:
tricks = list(df['trick'])

In [147]:
import ast
import pandas as pd
import requests

url = 'https://api.finance.naver.com/siseJson.naver'
params = {
        'symbol': '005935',  # 삼성전자
        'requestType': 1,
        'startTime': '20210101',
        'endTime': '20220101',
        'timeframe': 'day',  # 'week', 'month'
    }
res = requests.get(url, params=params)
data = res.text.strip()
stock_e = ast.literal_eval(data)  # 문자열을 배열로 변환
stock_e = pd.DataFrame(stock_e, columns=stock_e[0])
stock_e.drop(0, inplace=True)
stock_e['날짜'] = pd.to_datetime(stock_e['날짜'])
ohlcv = ['시가', '고가', '저가', '종가', '거래량']
stock_e[ohlcv] = stock_e[ohlcv].apply(pd.to_numeric)
stock_e['외국인소진율'] = stock_e['외국인소진율'].astype('float64')
stock_e['trick'] = '005935'
stock_e['change'] = (stock_e['종가'].pct_change(periods=1))*100

for n, i in enumerate(tricks):
    if n == 0: continue    
    
    params = {
        'symbol': f'{i}',
        'requestType': 1,
        'startTime': '20210101',
        'endTime': '20220101',
        'timeframe': 'day',  # 'week', 'month'
    }
    res = requests.get(url, params=params)
    data = res.text.strip()
    stock_es = ast.literal_eval(data)  # 문자열을 배열로 변환
    stock_es = pd.DataFrame(stock_es, columns=stock_es[0])
    stock_es.drop(0, inplace=True)
    stock_es['날짜'] = pd.to_datetime(stock_es['날짜'])
    ohlcv = ['시가', '고가', '저가', '종가', '거래량']
    stock_es[ohlcv] = stock_es[ohlcv].apply(pd.to_numeric)
    stock_es['외국인소진율' ] = stock_es['외국인소진율'].astype('float64')
    stock_es['trick'] = f'{i}'
    stock_es['change'] = (stock_es['종가'].pct_change(periods=1))*100
    stock_e = pd.concat([stock_e, stock_es], ignore_index=True)

In [148]:
stock_e

,날짜,시가,고가,저가,종가,거래량,외국인소진율,trick,change
0,2021-01-04,73700,75300,73600,74400,5455139,81.72,005935,NaN
1,2021-01-05,73600,75000,73600,75000,3870698,81.66,005935,0.806452
2,2021-01-06,75000,75600,74200,74300,4103763,81.49,005935,-0.933333
3,2021-01-07,74400,74800,74300,74300,3858695,81.41,005935,0.000000
4,2021-01-08,74700,80000,74300,77800,8225064,81.51,005935,4.710633
...,...,...,...,...,...,...,...,...,...
48059,2021-12-24,116845,117269,114470,116845,112747,44.46,048260,0.000000
48060,2021-12-27,116845,122695,116675,119982,269038,44.60,048260,2.684753
48061,2021-12-28,120660,122271,118541,122187,232636,44.45,048260,1.837776
48062,2021-12-29,122102,123119,119388,120491,170162,44.30,048260,-1.388036


In [165]:
stock = stock_e[stock_e['change'] > 20]

In [166]:
stock = pd.concat([stock, stock_e[stock_e['change'] < -20]], ignore_index=True) 

In [169]:
stock = stock.drop(columns=['시가', '고가', '저가', '종가', '거래량', '외국인소진율'])

In [173]:
columns = ['date', 'trick', 'change']
stock.columns = columns

In [174]:
stock

,date,trick,change
0,2021-01-21,066575,28.388017
1,2021-01-25,003495,29.985653
2,2021-06-07,034020,27.491125
3,2021-11-11,036570,29.917355
4,2021-05-06,011785,26.068376
5,2021-08-10,302440,29.677419
6,2021-04-21,000725,29.928448
7,2021-03-12,010145,29.812471
8,2021-03-15,010145,29.837097
9,2021-03-24,010145,29.983234


In [ ]:
import pandas_datareader.data as web
import datetime